In [1]:
import pandas as pd
import numpy as np
concrete_data = pd.read_csv('https://raw.githubusercontent.com/hsgeral/IBM-AI-Engineering-Professional-Certificate/main/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [2]:
concrete_data.shape

(1030, 9)

## Split data into predictors and target¶

In [3]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

In [4]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [5]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

Normalize the data by substracting the mean and dividing by the standard deviation.

In [6]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()

In [7]:
n_cols = predictors_norm.shape[1] # number of predictors

## Start the Neural Network part

In [8]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyterlab/con

Defining a function that creates a NN with 
- One hidden layer of num_nodes, and a ReLU activation function
- Use the adam optimizer and the mean squared error  as the loss function.

In [9]:
# define regression model
def regression_model_1hidden(num_nodes, num_inputs, num_outputs):
    # create model
    model = Sequential()
    model.add(Dense(num_nodes, activation='relu', input_shape=(num_inputs,)))
    model.add(Dense(num_outputs))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

## Train and Test the Network
- using the fit method to train and test the model at the same time,
- leaving out 30% of the data for validation 
- using 100 epochs to train the model.
- repeating 50 times to collect the 50 mean squared errors.

The model will set apart the 'validation_split' fraction of the training data, will not train on it, and will evaluate the loss and any model metrics on this data at the end of each epoch. 

In [10]:
num_nodes = 10
num_inputs = n_cols
num_outputs = 1
mse_list = [] # Initialize an empty list to store mean squared errors
for i in range(50):
    model = regression_model_1hidden(num_nodes, num_inputs, num_outputs) # build the model
    history=model.fit(predictors_norm, target, validation_split=0.3, epochs=100, verbose=0,shuffle=True) # fit the model
    mse_list.append(history.history['loss'][-1]) # get the mean square error after the 50th epoch
    print(i,mse_list[i])
# in each of the 50 repetitions the model was recreated otherwise the loss would constinously decrease as we were 
# fitting 50 * 100 epochs.
          
mean_mse = np.mean(mse_list)
std_mse = np.std(mse_list)
print (mean_mse,std_mse)

2024-06-20 17:45:55.167787: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
2024-06-20 17:45:55.172642: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2394310000 Hz
2024-06-20 17:45:55.173280: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x559c420c61a0 executing computations on platform Host. Devices:
2024-06-20 17:45:55.173324: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
2024-06-20 17:45:55.241008: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line fl

0 174.25839138163278
1 171.69834351771087
2 163.68931300233373
3 172.74255366861075
4 154.28376555211335
5 160.21677190222456
6 171.97485534625642
7 170.4557897115382
8 183.61531276411884
9 161.68744400172557
10 161.3955234522297
11 177.8328203050505
12 181.8460801927458
13 167.66296862894552
14 164.6607038945001
15 159.6069834546474
16 162.76049720034024
17 165.62572749081002
18 171.3133145992369
19 187.92825554412545
20 175.44137471658016
21 165.65675360352918
22 180.20994383749783
23 171.17757976600763
24 194.1400812495598
25 206.15319686656852
26 154.820465511158
27 180.08314015762815
28 166.10168263386424
29 161.10488629175788
30 170.95258203260445
31 194.8344863912765
32 170.2874266139016
33 196.79092350085466
34 181.01796674794528
35 164.31326306643334
36 184.93794127740742
37 159.27402715378759
38 180.66362469338512
39 172.4710437070976
40 189.75078524564407
41 163.0301900326627
42 164.052100193484
43 174.5478367481417
44 168.2339983343582
45 174.46402567997058
46 170.035559369